In [7]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 2522.4 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_10_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [11]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10,20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=994200, Wed Apr 15 14:04:42 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.332827764634900e+08 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.360203785810627e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.76931214e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.52463665e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.290441355614461e+08 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   30    570 3.471604500532487e+07 1.1e+00 1.96e+00  2e+00  2e+00 0:03.4
   64   1216 3.099059371168822e+06 1.1e+00 1.68e+00  2e+00  2e+00 0:07.5
  100   1900 -4.976485237770692e+07 1.2e+00 1.33e+00  1e+00  1e+00 0:11.7
  153   2907 -1.308267629267664e+08 1.2e+00 8.16e-01  8e-01  8e-01 0:17.7
  200   3800 -2.092129976879186e+08 1.2e+00 4.91e-01  5e-01  5e-01 0:23.0
  272   5168 -2.448409329414395e+08 1.2e+00 2.33e-01  2e-01  2e-01 0:31.1
  300   5700 -2.502930405435118e+08 1.2e+00 1.81e-01  2e-01  2e-01 0:34.3
  389   7391 -2.574570688657025e+08 1.2e+00 7.06e-02  7e-02  7e-02 0:44.4
  400   7600 -2.576576072206397e+08 1.2e+00 6.26e-02  6e-02  6e-02 0:45.6
  500   9500 -2.584482820376075e+08 1.2e+00 2.02e-02  2e-02  2e-02 0:56.7
  600  11400 -2.585287203425622e+08 1.2e+00 6.51e-03  6e-03  7e-03 1:07.7
  700  13300 -2.585378608089896e+08 1.2e+00 2.30e-03  2e-03  2e-03 1:18.8
  800  15200 -2.585389488288119e+08 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.56875172e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=156875172.076549, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteratio

termination on tolfun=1e-11 (Wed Apr 15 14:25:36 2020)
final/bestever f-value = 1.568752e+08 1.568752e+08
incumbent solution: [-4.99949032  2.62702631  3.03928176 -0.57436227  1.33420756  3.95019044
  2.76685091 -1.2266519  ...]
std deviations: [2.43483466 2.43530131 2.43478599 2.43538272 2.43503116 2.43486309
 2.43499653 2.43506724 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1063229, Wed Apr 15 14:25:36 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.282733797499069e+08 1.0e+00 2.43e+00  2e+00  2e+00 0:00.1
    2     38 1.189932475952747e+08 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

 8900 169100 -6.199299259845250e+08 8.7e+00 8.65e-09  6e-09  4e-08 16:15.8
 9000 171000 -6.199299259845251e+08 8.6e+00 8.86e-09  6e-09  4e-08 16:26.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 9100 172900 -6.199299259845250e+08 8.7e+00 8.18e-09  6e-09  4e-08 16:38.2
 9200 174800 -6.199299259845250e+08 8.7e+00 7.64e-09  5e-09  4e-08 16:49.3
 9300 176700 -6.199299259845250e+08 8.7e+00 6.50e-09  5e-09  3e-08 17:00.5


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-619929925.984525, sigma=6.62e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9318)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-619929925.984525, sigma=6.59e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9324)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-619929925.984525, sigma=6.48e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9333)
  ')')
/usr

 9400 178600 -6.199299259845252e+08 8.8e+00 5.80e-09  4e-09  3e-08 17:11.7
 9500 180500 -6.199299259845252e+08 8.8e+00 6.14e-09  4e-09  3e-08 17:22.8
 9600 182400 -6.199299259845252e+08 8.8e+00 5.44e-09  4e-09  3e-08 17:33.9
 9700 184300 -6.199299259845252e+08 8.8e+00 4.45e-09  3e-09  2e-08 17:45.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-619929925.984525, sigma=4.26e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9789)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-619929925.984525, sigma=4.16e-09).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=9794)
  ')')


 9800 186200 -6.199299259845252e+08 8.8e+00 4.05e-09  3e-09  2e-08 17:56.1
 9900 188100 -6.199299259845252e+08 8.9e+00 4.13e-09  3e-09  2e-08 18:07.2
10000 190000 -6.199299259845252e+08 9.0e+00 4.16e-09  3e-09  2e-08 18:18.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
10100 191900 -6.199299259845252e+08 9.0e+00 3.48e-09  2e-09  2e-08 18:29.4
10200 193800 -6.199299259845252e+08 9.0e+00 2.75e-09  2e-09  1e-08 18:40.6
10252 194788 -6.199299259845252e+08 9.0e+00 2.86e-09  2e-09  1e-08 18:46.4
termination on tolfunhist=1e-12 (Wed Apr 15 21:14:19 2020)
final/bestever f-value = -6.199299e+08 -6.199299e+08
incumbent solution: [ 5.          5.          5.         -5.          0.68647795  5.
 -5.         -5.         ...]
std deviations: [2.11834521e-09 2.13491705e-09 2.53241406e-09 2.41009594e-09
 2.95564808e-09 2.10641559e-09 6.85694002e-09 3.31453157e-09 ...]


In [13]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F10_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:200])
    SVM_Fun [i] = F10(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F10(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1.12086623e+08]), count=array([1])) 1.2438425145128602
SVM
ModeResult(mode=array([1.25038699e+08]), count=array([1])) 22661612.60422993
ELN
ModeResult(mode=array([1.79420276e+08]), count=array([1])) 50331091.98244641
